In [2]:
from fastapi import FastAPI
import pandas as pd
from datetime import datetime

In [3]:
df =  pd.read_csv("df_combinado.csv")
df

,id,title,genres_names,release_date,budget,revenue,runtime,vote_average,vote_count,production_companies_names,original_language,release_year,sentimiento,cast_procesado,crew_procesado
0,27205,Inception,"['Action', 'Thriller', 'Science Fiction', 'Mys...",2010-07-14,160000000.0,8.255328e+08,148.0,8.1,14075.0,"['Legendary Pictures', 'Warner Bros.', 'Syncopy']",en,2010,Positivo,"[{'nombre': 'Leonardo DiCaprio', 'personaje': ...","[{'nombre': 'Hans Zimmer', 'trabajo': 'Origina..."
1,24428,The Avengers,"['Science Fiction', 'Action', 'Adventure']",2012-04-25,220000000.0,1.519558e+09,143.0,7.4,12000.0,"['Paramount Pictures', 'Marvel Studios']",en,2012,Positivo,"[{'nombre': 'Robert Downey Jr.', 'personaje': ...","[{'nombre': 'Alan Silvestri', 'trabajo': 'Orig..."
2,293660,Deadpool,"['Action', 'Adventure', 'Comedy']",2016-02-09,58000000.0,7.831130e+08,108.0,7.4,11444.0,"['Twentieth Century Fox Film Corporation', 'Ma...",en,2016,Positivo,"[{'nombre': 'Ryan Reynolds', 'personaje': 'Wad...","[{'nombre': 'Robert Alonzo', 'trabajo': 'Stunt..."
3,157336,Interstellar,"['Adventure', 'Drama', 'Science Fiction']",2014-11-05,165000000.0,6.751200e+08,169.0,8.1,11187.0,"['Paramount Pictures', 'Legendary Pictures', '...",en,2014,Positivo,"[{'nombre': 'Matthew McConaughey', 'personaje'...","[{'nombre': 'Kendelle Elliott', 'trabajo': 'Ar..."
4,68718,Django Unchained,"['Drama', 'Western']",2012-12-25,100000000.0,4.253682e+08,165.0,7.8,10297.0,"['Columbia Pictures', 'The Weinstein Company']",en,2012,Positivo,"[{'nombre': 'Jamie Foxx', 'personaje': 'Django...","[{'nombre': 'Robert Rodriguez', 'trabajo': 'Th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24873,29054,Black Scorpion II: Aftershock,"['Comedy', 'Action', 'Adventure', 'Science Fic...",1997-05-13,0.0,0.000000e+00,85.0,5.7,3.0,['Concorde-New Horizons'],en,1997,Neutral,"[{'nombre': 'Joan Severance', 'personaje': 'Da...","[{'nombre': 'Roger Corman', 'trabajo': 'Execut..."
24874,60457,Michael Jordan: An American Hero,['Drama'],1999-04-18,0.0,0.000000e+00,0.0,5.2,3.0,[],en,1999,Neutral,"[{'nombre': 'Michael Jace', 'personaje': 'Mich...","[{'nombre': 'Alan Metzger', 'trabajo': 'Direct..."
24875,105073,The Shadow Strikes,['Mystery'],1937-10-29,0.0,0.000000e+00,62.0,3.7,3.0,['Colony Pictures'],en,1937,Negativo,"[{'nombre': 'Rod La Rocque', 'personaje': 'Lam...","[{'nombre': 'Lynn Shores', 'trabajo': 'Directo..."
24876,85827,Island of the Blue Dolphins,"['Family', 'Drama']",1964-09-10,0.0,0.000000e+00,93.0,5.7,3.0,"['Universal Pictures', 'Robert B. Radnitz Prod...",en,1964,Neutral,"[{'nombre': 'Celia Kaye', 'personaje': 'Karana...","[{'nombre': 'James B. Clark', 'trabajo': 'Dire..."


#### def cantidad_filmaciones_mes( Mes ): Esta función cuenta las películas lanzadas en un mes específico.

In [4]:
# Crear la instancia de la aplicación
app = FastAPI()

@app.get("/cantidad_filmaciones_mes/{mes}")
def cantidad_filmaciones_mes(mes: str):
    # Convertir el mes a número
    meses = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    mes_num = meses.get(mes.lower())
    
    if mes_num is None:
        return {"error": "Mes no válido"}
    
    # Filtrar el DataFrame por el mes
    cantidad = df[df['release_date'].dt.month == mes_num].shape[0]
    
    return {"message": f"{cantidad} cantidad de películas fueron estrenadas en el mes de {mes.capitalize()}"}



#### ccantidad_filmaciones_dia( Dia ):  Esta función cuenta las películas lanzadas en un día específico de la semana.

In [5]:
@app.get("/cantidad_filmaciones_dia/{dia}")
def cantidad_filmaciones_dia(dia: str):
    dias_semana = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3,
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    dia_num = dias_semana.get(dia.lower())
    
    if dia_num is None:
        return {"error": "Día no válido"}
    
    # Filtrar el DataFrame por el día de la semana
    cantidad = df[df['release_date'].dt.dayofweek == dia_num].shape[0]
    
    return {"message": f"{cantidad} cantidad de películas fueron estrenadas en los días {dia.capitalize()}"}


#### score_titulo( titulo_de_la_filmación): Esta función devuelve el título, año y score de una película.

In [6]:
@app.get("/score_titulo/{titulo}")
def score_titulo(titulo: str):
    pelicula = df[df['title'].str.lower() == titulo.lower()]
    
    if pelicula.empty:
        return {"error": "Película no encontrada"}
    
    titulo = pelicula['title'].values[0]
    anio = pelicula['release_year'].values[0]
    score = pelicula['vote_average'].values[0]
    
    return {"message": f"La película {titulo} fue estrenada en el año {anio} con un score de {score}"}


##### votos_titulo( titulo_de_la_filmación ): Esta función devuelve el título, cantidad de votos y promedio de votación de una película. Verifica que tenga al menos 2000 votos.

In [7]:
@app.get("/votos_titulo/{titulo}")
def votos_titulo(titulo: str):
    pelicula = df[df['title'].str.lower() == titulo.lower()]
    
    if pelicula.empty:
        return {"error": "Película no encontrada"}
    
    titulo = pelicula['title'].values[0]
    anio = pelicula['release_year'].values[0]
    votos = pelicula['vote_count'].values[0]
    promedio = pelicula['vote_average'].values[0]
    
    if votos < 2000:
        return {"message": f"La película {titulo} no cumple con la cantidad mínima de valoraciones (2000)"}
    
    return {"message": f"La película {titulo} fue estrenada en el año {anio}. Tiene un total de {votos} valoraciones, con un promedio de {promedio}"}


#### get_actor( nombre_actor ): Esta función calcula el éxito del actor en función de su retorno y número de participaciones.

In [8]:
@app.get("/get_actor/{nombre_actor}")
def get_actor(nombre_actor: str):
    participaciones = df[df['cast_procesado'].str.contains(nombre_actor, case=False, na=False)]
    
    if participaciones.empty:
        return {"error": "Actor no encontrado"}
    
    cantidad_peliculas = participaciones.shape[0]
    retorno_total = participaciones['revenue'].sum() / participaciones['budget'].sum()
    retorno_promedio = retorno_total / cantidad_peliculas
    
    return {"message": f"El actor {nombre_actor} ha participado de {cantidad_peliculas} filmaciones, con un retorno total de {retorno_total} y un promedio de {retorno_promedio:.2f} por filmación"}


#### get_director( nombre_director ): Esta función devuelve el éxito del director, junto con información detallada de cada película.

In [9]:
@app.get("/get_director/{nombre_director}")
def get_director(nombre_director: str):
    direcciones = df[df['crew_procesado'].str.contains(f"'nombre': '{nombre_director}'", case=False, na=False)]
    
    if direcciones.empty:
        return {"error": "Director no encontrado"}
    
    peliculas = []
    for _, row in direcciones.iterrows():
        retorno_individual = row['revenue'] / row['budget'] if row['budget'] > 0 else 0
        pelicula_info = {
            "titulo": row['title'],
            "fecha_lanzamiento": row['release_date'],
            "retorno": retorno_individual,
            "costo": row['budget'],
            "ganancia": row['revenue'] - row['budget']
        }
        peliculas.append(pelicula_info)
    
    retorno_total = direcciones['revenue'].sum() / direcciones['budget'].sum() if direcciones['budget'].sum() > 0 else 0
    
    return {
        "message": f"El director {nombre_director} tiene un retorno total de {retorno_total}",
        "peliculas": peliculas
    }
